# Theano and Lasagne


### Installation instructions

Istall via pip by executing the following commands:

<code>pip install --upgrade https://github.com/Theano/Theano/archive/master.zip
pip install --upgrade https://github.com/Lasagne/Lasagne/archive/master.zip<\code>

or go to this page for further clarifications:

http://lasagne.readthedocs.org/en/latest/user/installation.html

# Warming up: compare timings
* Implement a function that computes the sum of squares of numbers from 0 to N
* Use numpy or python
* An array of numbers 0 to N - numpy.arange(N)

In [ ]:
#no numpy, just python
def sum_squares(N):
    return sum([x**2 for x in range(N)])

%timeit -n 1 sum_squares(10**6)

In [ ]:
#the same computation with numpy
import numpy as np
def sum_squares(N):
    return (np.arange(N)**2).sum()

%timeit -n 1 sum_squares(10**6)

In [ ]:
#the same computation with theano
import theano
import theano.tensor as T

#input variable
N = T.scalar("number of terms", dtype='int32')

#a recipe how to produce sum of squares of arange of N given N
result = (T.arange(N)**2).sum()

#compilation of function
sum_squares = theano.function(inputs = [N],outputs=result)

%timeit -n 1 sum_squares(10**6)

# Part One
# Theano: key concepts

## 1 Symbolic variables

In [ ]:
eps = T.scalar('example scalar', dtype='float32')
v = T.vector()#dtype = theano.config.floatX by default
x = T.tensor4()

## 2 Expressions

In [ ]:
# Transformations
two_times_v = 2 * v
elemetwise_sin = T.sin(v)
x_normalized = (x - x.mean(axis=0)) / (x.std(axis=0) + eps)

In [ ]:
theano.printing.debugprint(two_times_v)

## 3 Functions

With variables and expressions, you can define a recipe for computation, but you need a function to actually compute something

In [ ]:
#symbolic variables are placeholders for input values in our function
a = T.vector('first input')
b = T.vector('second input')

#theano expressions are a scheme of computation process
c = a + b

inputs = [a, b]
outputs = c

#we compile a function
#theano requires function inputs to be in a list, evem if the function has single input
f = theano.function(inputs, outputs)

#now we can perform an actual computation
print(f(np.ones(5), np.ones(5)))

Frequent problem: numpy uses float64 by default, theano - float32.
Feeding float64 inputs to the function compiled for float32 will produce an error.
There are several ways to overcome this problem
* make numpy to use float32 with <code>dtype=np.float32</code>, or explicitly convert your data via <code>.astype(np.float32) </code>
* set type of your input variables to 'float64'
* compile function with flag <code>allow_input_downcast=True</code>

## 4 Shared variables

* the inputs and transformations only exist when function is called

* shared variables always stay in memory, and can be shared between multiple functions

* hybrid symbolic and non-symbolic variables
 
* a perfect place to store network weights

In [ ]:
#creating shared variable
shared_vector = theano.shared(np.ones(5))

#shared variable looks like a symbolic variable
print(shared_vector)

In [ ]:
#but it can be evaluated because it's got a value inside
print(shared_vector.get_value())

#within symbolic graph you use them just as any other inout or transformation, no "get value" needed

In [ ]:
#setting new value
shared_vector.set_value(np.arange(4))

print(shared_vector.get_value())

#Note that the vector changed shape
#This is entirely allowed... unless your graph is hard-wired to work with some fixed shape

## 4.1 Updates

* updates are a way to change shared variable value after a function call

* update is a dictionary of a form {shared_variable : an expression for new value} which is has to be provided when function is compiled

That's how it works:

In [ ]:
# Accumulator function
input_scalar = T.scalar(name='input',dtype='float64')
accum = theano.shared(0.)

inputs = [input_scalar]
outputs = []
my_updates = {accum: accum + input_scalar}

f = theano.function(inputs, outputs, updates=my_updates)

In [ ]:
print accum.get_value()
f(1)
print accum.get_value()
f(1)
print accum.get_value()
f(137)
print accum.get_value()

## 5 Special feature: symbolic differentiation
* Theano can compute derivatives and gradients automatically
* Derivatives are computed symbolically, not numerically

Limitations:
* You can only compute a gradient of a __scalar__ over one or several scalars,  vector or tensors
* A transformation has to have float32 or float64 dtype throughout the whole computation graph
 * derivative over an integer has no mathematical sense

In [ ]:
my_scalar = T.scalar(name='input',dtype='float64')

scalar_squared = T.sum(my_scalar**2)

#a derivative of v_squared by my_vector
derivative = T.grad(scalar_squared, my_scalar)

f = theano.function([my_scalar], scalar_squared)
grad = theano.function([my_scalar], derivative) 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

x = np.linspace(-3, 3)
x_squared = list(map(f, x))
d_x_squared = list(map(grad,x))

plt.plot(x, x_squared, label="$x^2$")
plt.plot(x, d_x_squared, label="derivative")
plt.legend()
plt.show()

## Exercise

In [ ]:
my_vector = T.vector('float64')

#Compute the gradient of the next weird function over my_scalar and my_vector
#warning! Trying to understand the meaning of that function may result in permanent brain damage

weird_psychotic_function = ((my_vector+my_scalar)**(1+T.var(my_vector)) +1./T.arcsinh(my_scalar)).mean()/(my_scalar**2 +1) + 0.01*T.sin(2*my_scalar**1.5)*(T.sum(my_vector)* my_scalar**2)*T.exp((my_scalar-4)**2)/(1+T.exp((my_scalar-4)**2))*(1.-(T.exp(-(my_scalar-4)**2))/(1+T.exp(-(my_scalar-4)**2)))**2


der_by_scalar,der_by_vector = <student.compute_grad_over_scalar_and_vector()>


compute_weird_function = theano.function([my_scalar,my_vector],weird_psychotic_function)
compute_der_by_scalar = theano.function([my_scalar,my_vector],der_by_scalar)


In [ ]:
#Plotting your derivative
vector_0 = [1,2,3]

scalar_space = np.linspace(0,7)

y = [compute_weird_function(x,vector_0) for x in scalar_space]
plt.plot(scalar_space,y,label='function')
y_der_by_scalar = [compute_der_by_scalar(x,vector_0) for x in scalar_space]
plt.plot(scalar_space,y_der_by_scalar,label='derivative')
plt.grid();plt.legend()


## Exercise 2: logistic regression

Implement the regular logistic regression training algorithm

Tips:
* Weights fit in as a shared variable
* X and y are potential inputs
* Compile 2 functions:
 * train_function(X,y) - returns error and computes weights' new values __(through updates)__
 * predict_fun(X) - just computes probabilities ("y") given data
 
 
We shall train on a two-class MNIST dataset
* please note that target y are {0,1} and not {-1,1} as in some formulae

In [ ]:
from sklearn.datasets import load_digits
mnist = load_digits(2)

X,y = mnist.data, mnist.target


print ("y [shape - %s]:"%(str(y.shape)),y[:10])

print ("X [shape - %s]:"%(str(X.shape)))
print (X[:3])
print (y[:10])

In [ ]:
# inputs and shareds
shared_weights = <student.code_me()>
input_X = <student.code_me()>
input_y = <student.code_me()>

In [ ]:
predicted_y = <predicted probabilities for input_X>


loss = <logistic loss (scalar, mean over sample)>


grad = <gradient of loss over model weights>


updates = {
    shared_weights: <new weights after gradient step>
}

In [ ]:

train_function = <compile function that takes X and y, returns log loss and updates weights>

predict_function = <compile function that takes X and computes probabilities of y>

In [ ]:
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [ ]:
from sklearn.metrics import roc_auc_score

for i in range(5):
    loss_i = train_function(X_train,y_train)
    print ("loss at iter %i:%.4f"%(i,loss_i))
    print ("train auc:",roc_auc_score(y_train,predict_function(X_train)))
    print ("test auc:",roc_auc_score(y_test,predict_function(X_test)))

    
print ("resulting weights:")
plt.imshow(shared_weights.get_value().reshape(8,-1))
plt.colorbar()

# Part Two: Lasagne
* lasagne is a library for neural network building and training, built on top of theano

For a demo we shall solve the same MNIST digit recognition problem
* image size: 28x28
* 10 different digits
* 50k samples

In [ ]:
from mnist import load_dataset
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()

print X_train.shape, y_train.shape
print X_val.shape, y_val.shape
print X_test.shape, y_test.shape

Defining network architecture

In [ ]:
import lasagne

X = T.tensor4()
y = T.vector(dtype='int32')

input_layer = lasagne.layers.InputLayer(shape=[None, 1, 28, 28], input_var=X)
#None means arbitrary number and only works for the first axis
#if you don't create input variable, input layer will create it automatically

#fully connected layer, that takes input layer and applies 50 neurons to it.
# nonlinearity here is sigmoid as in logistic regression
# you can give a name to each layer (optional)
dense_1 = lasagne.layers.DenseLayer(input_layer,num_units=50,
                                   nonlinearity = lasagne.nonlinearities.sigmoid,
                                   name = "hidden_layer")

#fully connected output layer that takes dense_1 as input and has 10 neurons (1 for each digit)
#We use softmax nonlinearity to make probabilities add up to 1
output_layer = lasagne.layers.DenseLayer(dense_1, num_units = 10,
                                        nonlinearity = lasagne.nonlinearities.softmax,
                                        name='output_layer')

In [ ]:
#network prediction (theano-transformation)
prediction = lasagne.layers.get_output(output_layer)

In [ ]:
#all network weights (shared variables)
weights = lasagne.layers.get_all_params(output_layer)
print (weights)

In principle, we can go the hard way, and do everyting with theano only
* define loss function manually
* compute error gradient over all weights
* define updates
* But that's a lot of work and life's short

Instead, we shall use Lasagne builtins

In [ ]:
#Mean categorical crossentropy as a loss function - similar to logistic loss but for multiclass targets
loss = lasagne.objectives.categorical_crossentropy(prediction, y).mean()

#prediction accuracy
accuracy = lasagne.objectives.categorical_accuracy(prediction, y).mean()

#This function computes gradient AND composes weight updates just like you did earlier
updates_sgd = lasagne.updates.sgd(loss, weights, learning_rate=0.01)

#function that computes loss and updates weights
train_fn = theano.function([X, y], [loss, accuracy], updates=updates_sgd)

#function that just computes accuracy
accuracy_fn = theano.function([X, y], accuracy)

### That's all, now let's train it!
* We got a lot of data, so it's recommended that you use SGD
* So let's implement a function that splits the training sample into minibatches

In [ ]:
# An auxilary function that returns mini-batches for neural network training

#Parameters
# X - a tensor of images with shape (many, 1, 28, 28), e.g. X_train
# y - a vector of answers for corresponding images e.g. Y_train
#batch_size - a single number - the intended size of each batches

#What do need to implement
# 1) Shuffle data
# - You have to shuffle X and y the same way, or the correspondence between X_i and y_i will be broken
# 3) Split data into minibatches of batch_size
# - If data size is not a multiple of batch_size, make one last batch smaller.
# 4) return a list (or an iterator) of pairs
# - (images, corresponding labels)

def iterate_minibatches(X, y, batch_size):
    raise NotImplementedError

# Training loop

In [ ]:
import time

num_epochs = 100 #amount of passes through the data

batch_size = 50 #number of samples processed at each function call

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train,batch_size):
        inputs, targets = batch
        train_err_batch, train_acc_batch = train_fn(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fn(inputs, targets)
        val_batches += 1

    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(
        train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

In [ ]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fn(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 99:
    print ("Achievement unlocked: 80lvl Warlock!")
else:
    print ("We need more magic!")

# Exercise: make a better network


* The quest is to create a network that gets at least 99% at test set

## Tips on what can be done:

   * more layers!
   * more neurons!  
   * ok, forget about more neurons. If you use fully-connected layers, it will certainly lead to overfitting. Use convolutions instead
   * `network = lasagne.layers.Conv2DLayer(prev_layer,`
    `                       num_filters = n_neurons,`
    `                        filter_size = (filter width, filter height),`
    `                        nonlinearity = some_nonlinearity)`
   * Warning! Training convolutional networks can take long without GPU
   
 * Regularize to prevent overfitting
   * Add some L2 weight norm to the loss function, theano will do the rest
   * Can be done manually or via - http://lasagne.readthedocs.org/en/latest/modules/regularization.html
   
 * Better optimization method - rmsprop, nesterov_momentum, adadelta, adagrad and so on.
   * Converge faster and sometimes reach better optima
   * It might make sense to tweak learning rate, other learning parameters, batch size and number of epochs   
   
 * Dropout - to prevent overfitting
   * `lasagne.layers.DropoutLayer(prev_layer, p=probability_to_zero_out)`
   
 * Plenty other layers and architectures
   * http://lasagne.readthedocs.org/en/latest/modules/layers.html
   * batch normalization, pooling, etc
   
 * Nonlinearities in the hidden layers
   * tanh, relu, leaky relu, etc